In [3]:
from recomendations import critics

persons = list(critics.keys())
unique_films = set()
for person in persons:
    unique_films.update(critics[person])


# Eucledian distance

In [4]:
from math import sqrt

def sim_distance(prefs, person1, person2):
    si = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1
    if not si:
        return 0
    sum_of_squares = sum([pow(prefs[person1][item] - prefs[person2][item], 2) for item in prefs[person1] if item in prefs[person2]])
    return 1 / (1 + sum_of_squares)
print(sim_distance(critics, 'Lisa Rose', 'Gene Seymour'))

0.14814814814814814


# Pearson correlation quotient

Использование линии наилучшего приближения. Если один критик склонен выставлять более высокие оценки, чем другой, то идеальная корреляция все равно возможна при условии, что разница в оценках постоянна. Метод евклидова расстояния в этом случае выдал бы результат, что критики не похожи, поскольку один всегда оказывается строже другого, несмотря на то что их вкусы, по существу, очень сходны. 
В зависимости от конкретного приложения такое поведение может вас 
устраивать или нет.

Программа для вычисления коэффициента корреляции Пирсона сначала  находит  фильмы,  оцененные  обоими  критиками,  и  вычисляет 
сумму  и  сумму  квадратов  выставленных  ими  оценок,  а  также  сумму 
произведений  оценок.  На  последнем  этапе  найденные  значения  используются для вычисления коэффициента корреляции; этот код выделен в листинге ниже полужирным шрифтом. В отличие от евклидовой метрики, эта формула интуитивно не так очевидна.

$$ r_{xy} = \frac{\sum_{i=1}^{m}(x_i - \overline{x})(y_i - \overline{y})}{\sqrt{\sum_{i=1}^{m}(x_i - \overline{x})^{2}\sum_{i=1}^{m}(y_i - \overline{y})^{2}}} $$

In [26]:
def sim_pearson(prefs, p1, p2):
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    n = len(si)
    if n == 0:
        return 0
    # Сумма всех предпочтений
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    # Сумма квадратов предпочтений
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])

    # Сумма произведений общих предпочтений
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])


    # Вычисление коэфициента Пирсона
    num = pSum - (sum1 * sum2 / float(n))
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0

    return float(num) / float(den)

print(sim_pearson(critics, 'Mick LaSalle', 'Gene Seymour'))


0.41176470588235276


In [34]:
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other) for other in prefs if other is not person]
    
    scores.sort()
    scores.reverse()
    return scores[0:n]
print(topMatches(critics, 'Toby', n=3, similarity=sim_pearson))
print(topMatches(critics, 'Toby', n=3, similarity=sim_distance))

[(0.9912407071619299, 'Lisa Rose'), (0.9244734516419049, 'Mick LaSalle'), (0.8934051474415647, 'Claudia Puig')]
[(0.3076923076923077, 'Mick LaSalle'), (0.2857142857142857, 'Michael Phillips'), (0.23529411764705882, 'Claudia Puig')]


Методы ранжирования: берем критиков, умножаем его оценку подобия с пользователем на оценку, которую он поставил каждому фильму, выбираем фильмы с максимальными баллами.

In [49]:
def getRecommendations(prefs, person, sim_func=sim_pearson):
    totals = {}
    simSums = {}
    for other in prefs:
        if other == person:
            continue
        sim = sim_func(prefs, person, other)
        if sim <= 0:
            continue
        for item in prefs[other]:
            # оцениваем непросмотренные фильмы
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                
                simSums.setdefault(item, 0)
                simSums[item] += sim
    print(totals)
    rankings = [(total / simSums[item], item) for item, total in totals.items()]
    rankings.sort()
    rankings.reverse()
    return rankings

getRecommendations(critics, 'Toby')

{'The Night Listener': 12.89975185847269, 'Just My Luck': 8.074754105841562, 'Lady in the Water': 8.383808341404684}


[(3.3477895267131013, 'The Night Listener'),
 (2.832549918264162, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]